<h1>Cloud Data Management Project: COVID19 analysis</h1>
<h2>730030 Vladi VALSECCHI & 730113 Gianluca MOLTENI</h2>
<p>The project aims to analyze the SARS-CoV-19 pandemic, commonly known as Coronavirus or Covid19 for short.
Through queries on various datasets, relating to the infection and the vaccination campaign, we will investigate different consequences of this pandemic.</p>

In [ ]:
#install pyspark
!pip install pyspark

In [ ]:
#Import
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [ ]:
#Datasets urls
url_region="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
url_vaccine="https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"

In [ ]:
#Create pandas data frame from url
pdf_r=pd.read_csv(url_region)
pdf_v=pd.read_csv(url_vaccine)

In [ ]:
#fix regions dataset (keep only date yyyy-mm-dd)
pdf_r['data']=pdf_r['data'].str[:10] #keep only the first 10 characters
#fix vaccine dataset (Provincia Autonoma di Trento and Bolzano from region code 4 to 21 and 22)
unique_areas=pdf_v.nome_area.unique()#create unique area names list
pdf_v.loc[pdf_v.nome_area == unique_areas[11], "codice_regione_ISTAT"] = np.int64(21)#change unique area name region code in position 11 from 4 to 21
pdf_v.loc[pdf_v.nome_area == unique_areas[12], "codice_regione_ISTAT"] = np.int64(22)#change unique area name region code in position 12 from 4 to 22

In [ ]:
#Spark setup
spark = SparkSession.builder.appName("Query").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
#Create spark data frame from pandas data frame
dpc_covid19_ita_regioni = spark.createDataFrame(pdf_r)#create spark dataframe for dpc_covid19_ita_regioni
somministrazioni_vaccini_latest=spark.createDataFrame(pdf_v)#create spark dataframe for somministrazioni_vaccini_latest

In [ ]:
#Create temporary views that will be used in the queries
dpc_covid19_ita_regioni.createOrReplaceTempView("base")
somministrazioni_vaccini_latest.createOrReplaceTempView("base2")

In [ ]:
dpc_covid19_ita_regioni.printSchema()

root
 |-- data: string (nullable = true)
 |-- stato: string (nullable = true)
 |-- codice_regione: long (nullable = true)
 |-- denominazione_regione: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- ricoverati_con_sintomi: long (nullable = true)
 |-- terapia_intensiva: long (nullable = true)
 |-- totale_ospedalizzati: long (nullable = true)
 |-- isolamento_domiciliare: long (nullable = true)
 |-- totale_positivi: long (nullable = true)
 |-- variazione_totale_positivi: long (nullable = true)
 |-- nuovi_positivi: long (nullable = true)
 |-- dimessi_guariti: long (nullable = true)
 |-- deceduti: long (nullable = true)
 |-- casi_da_sospetto_diagnostico: double (nullable = true)
 |-- casi_da_screening: double (nullable = true)
 |-- totale_casi: long (nullable = true)
 |-- tamponi: long (nullable = true)
 |-- casi_testati: double (nullable = true)
 |-- note: string (nullable = true)
 |-- ingressi_terapia_intensiva: double (nullable = true)


In [ ]:
somministrazioni_vaccini_latest.printSchema()

root
 |-- data_somministrazione: string (nullable = true)
 |-- fornitore: string (nullable = true)
 |-- area: string (nullable = true)
 |-- fascia_anagrafica: string (nullable = true)
 |-- sesso_maschile: long (nullable = true)
 |-- sesso_femminile: long (nullable = true)
 |-- prima_dose: long (nullable = true)
 |-- seconda_dose: long (nullable = true)
 |-- pregressa_infezione: long (nullable = true)
 |-- codice_NUTS1: string (nullable = true)
 |-- codice_NUTS2: string (nullable = true)
 |-- codice_regione_ISTAT: long (nullable = true)
 |-- nome_area: string (nullable = true)



********************************************************************************
Query 0: show last day recorded for each region

SELECT * --select all

FROM base --from dpc_covid19_ita_regioni

ORDER BY data DESC --order from the biggest to the smallest value

LIMIT 21 --limit to 21 to get only 1 result for each region

In [ ]:
query_0=spark.sql("SELECT * FROM base ORDER BY data DESC LIMIT 21")
query_0.show(21)

+----------+-----+--------------+---------------------+-----------+------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+----------------------------+-----------------+-----------+--------+------------+--------------------+--------------------------+---------+--------------------+-------------------------------+--------------------------------------+-----------------------+------------------------------+-------------+-------------+
|      data|stato|codice_regione|denominazione_regione|        lat|              long|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|casi_da_sospetto_diagnostico|casi_da_screening|totale_casi| tamponi|casi_testati|                note|ingressi_terapia_intensiva|note_test|           note_casi|totale_positivi_t

********************************************************************************
Query 1: show top 5 regions for most cases

SELECT

MAX(totale_casi) AS top_5_regions_for_most_cases, --get max value of 
totale_casi alias top_5_regions_for_most_cases

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last 
date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(totale_casi) DESC  --order from biggest to smallest value

LIMIT 5 --limit to 5 to get only the first 5 results

In [ ]:
query_1=spark.sql("SELECT MAX(totale_casi) AS top_5_regions_for_most_cases, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MAX(totale_casi) DESC LIMIT 5")
query_1.show()

+----------------------------+--------------+
|top_5_regions_for_most_cases|        region|
+----------------------------+--------------+
|                      843891|     Lombardia|
|                      426743|        Veneto|
|                      426122|      Campania|
|                      387789|Emilia-Romagna|
|                      363360|      Piemonte|
+----------------------------+--------------+



********************************************************************************
Query 2: show top 5 regions for less cases

SELECT 

MIN(totale_casi) AS top_5_regions_for_less_cases, --get min value of totale_casi alias top_5_regions_for_less_cases

denominazione_regione AS region  --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21)  --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(totale_casi) ASC --order from smallest to biggest value 

LIMIT 5 --limit to 5 to get only the first 5 results

In [ ]:
query_2=spark.sql("SELECT MIN(totale_casi) AS top_5_regions_for_less_cases, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(totale_casi) ASC LIMIT 5")
query_2.show()

+----------------------------+-------------+
|top_5_regions_for_less_cases|       region|
+----------------------------+-------------+
|                       11703|Valle d'Aosta|
|                       13750|       Molise|
|                       27044|   Basilicata|
|                       45828|  P.A. Trento|
|                       56958|       Umbria|
+----------------------------+-------------+



********************************************************************************
Query 3: show average number of cases daily for each region

SELECT 

AVG(nuovi_positivi) AS avg_new_cases_daily, --get average value of nuovi_positivi alias avg_new_cases_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(nuovi_positivi) DESC --order from biggest to smallest value


In [ ]:
query_3=spark.sql("SELECT AVG(nuovi_positivi) AS avg_new_cases_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(nuovi_positivi) DESC")
query_3.show(21)

+-------------------+--------------------+
|avg_new_cases_daily|              region|
+-------------------+--------------------+
| 1667.8162055335968|           Lombardia|
|  843.3636363636364|              Veneto|
|  842.0553359683795|            Campania|
|  769.0889328063241|      Emilia-Romagna|
|  723.2213438735178|            Piemonte|
|  686.6363636363636|               Lazio|
|  501.8102766798419|              Puglia|
| 484.34584980237156|             Toscana|
| 462.47233201581025|             Sicilia|
|  213.4683794466403|Friuli Venezia Gi...|
|  205.6304347826087|              Marche|
| 204.42094861660078|             Liguria|
|  148.9683794466403|             Abruzzo|
| 137.10474308300394|            Calabria|
| 124.43873517786561|        P.A. Bolzano|
| 114.00197628458498|            Sardegna|
| 112.56521739130434|              Umbria|
|  89.80434782608695|         P.A. Trento|
|  53.47826086956522|          Basilicata|
| 27.181818181818183|              Molise|
|  23.07509

********************************************************************************
Query 4: show top 5 regions for most diagnostic tests done

SELECT 

MAX(tamponi) AS top_5_regions_for_most_DiagnosticTests_done, --get max value of tamponi alias top_5_regions_for_most_DiagnosticTests_done

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(tamponi) DESC --order from biggest to smallest value

LIMIT 5 --limit to 5 to get only the first 5 results

In [ ]:
query_4=spark.sql("SELECT MAX(tamponi) AS top_5_regions_for_most_DiagnosticTests_done, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MAX(tamponi) DESC LIMIT 5")
query_4.show()

+-------------------------------------------+--------------+
|top_5_regions_for_most_DiagnosticTests_done|        region|
+-------------------------------------------+--------------+
|                                   11963426|     Lombardia|
|                                    8964062|        Veneto|
|                                    7902542|         Lazio|
|                                    6924098|Emilia-Romagna|
|                                    5433896|      Campania|
+-------------------------------------------+--------------+



********************************************************************************
Query 5: show top 5 regions for less diagnostic tests done

SELECT 

MIN(tamponi) AS top_5_regions_for_less_DiagnosticTests_done, --get min value of tamponi alias top_5_regions_for_less_DiagnosticTests_done

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(tamponi) ASC --order from smallest to biggest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_5=spark.sql("SELECT MIN(tamponi) AS top_5_regions_for_less_DiagnosticTests_done, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(tamponi) ASC LIMIT 5")
query_5.show()

+-------------------------------------------+-------------+
|top_5_regions_for_less_DiagnosticTests_done|       region|
+-------------------------------------------+-------------+
|                                     144342|Valle d'Aosta|
|                                     244525|       Molise|
|                                     399971|   Basilicata|
|                                     884608|  P.A. Trento|
|                                     956364|     Calabria|
+-------------------------------------------+-------------+



********************************************************************************
Query 6: show average number of diagnostic tests done daily for each region

SELECT 

AVG(tamponi) AS avg_new_DiagnosticTests_daily, --get average value of tamponi alias avg_new_DiagnosticTests_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(tamponi) DESC --order from the biggest to the smallest value


In [ ]:
query_6=spark.sql("SELECT AVG(tamponi) AS avg_new_DiagnosticTests_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(tamponi) DESC")
query_6.show(21)

+-----------------------------+--------------------+
|avg_new_DiagnosticTests_daily|              region|
+-----------------------------+--------------------+
|            4236973.332015811|           Lombardia|
|            3225902.796442688|              Veneto|
|           2509649.9980237153|               Lazio|
|           2399146.1264822134|      Emilia-Romagna|
|           1778264.9308300395|            Campania|
|           1743723.0928853755|             Toscana|
|           1686678.3577075098|            Piemonte|
|           1486382.3932806323|             Sicilia|
|            939772.3023715415|              Puglia|
|            815072.2865612648|Friuli Venezia Gi...|
|            637635.3478260869|             Liguria|
|            566370.3656126482|             Abruzzo|
|           502037.25889328064|        P.A. Bolzano|
|            498379.9743083004|              Marche|
|            492159.8577075099|              Umbria|
|           470909.86363636365|            Sar

********************************************************************************
Query 7: show top 5 regions for most deaths

SELECT 

MAX(deceduti) AS top_5_regions_for_most_deaths, --get max value of deceduti alias top_5_regions_for_most_deaths

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(deceduti) DESC --order from the biggest to the smallest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_7=spark.sql("SELECT MAX(deceduti) AS top_5_regions_for_most_deaths, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MAX(deceduti) DESC LIMIT 5")
query_7.show()

+-----------------------------+--------------+
|top_5_regions_for_most_deaths|        region|
+-----------------------------+--------------+
|                        33802|     Lombardia|
|                        13268|Emilia-Romagna|
|                        11698|      Piemonte|
|                        11623|        Veneto|
|                         8380|         Lazio|
+-----------------------------+--------------+



********************************************************************************
Query 8: show top 5 regions for less deaths

SELECT 

MIN(deceduti) AS top_5_regions_for_less_deaths, --get min value of deceduti alias top_5_regions_for_less_deaths

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(deceduti) ASC --order from smallest to the biggest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_8=spark.sql("SELECT MIN(deceduti) AS top_5_regions_for_less_deaths, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(deceduti) ASC LIMIT 5")
query_8.show()

+-----------------------------+-------------+
|top_5_regions_for_less_deaths|       region|
+-----------------------------+-------------+
|                          474|Valle d'Aosta|
|                          492|       Molise|
|                          591|   Basilicata|
|                         1182| P.A. Bolzano|
|                         1236|     Calabria|
+-----------------------------+-------------+



********************************************************************************
Query 9: show average number of deaths daily for each region

SELECT 

AVG(deceduti) AS avg_deaths_daily, --get average value of deceduti alias avg_deaths_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(deceduti) DESC --order from the biggest to the smallest value


In [ ]:
query_9=spark.sql("SELECT AVG(deceduti) AS avg_deaths_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(deceduti) DESC")
query_9.show(21)

+------------------+--------------------+
|  avg_deaths_daily|              region|
+------------------+--------------------+
|21201.533596837944|           Lombardia|
| 6920.652173913043|      Emilia-Romagna|
| 6303.365612648221|            Piemonte|
| 5273.818181818182|              Veneto|
|3195.7509881422925|               Lazio|
| 2871.195652173913|             Toscana|
|2508.0573122529645|            Campania|
|2390.6383399209485|             Liguria|
|2305.9644268774705|              Puglia|
|2113.2509881422925|             Sicilia|
|1546.9841897233202|              Marche|
|1476.3517786561265|Friuli Venezia Gi...|
|1074.0197628458498|             Abruzzo|
| 737.7094861660079|         P.A. Trento|
| 592.9229249011858|        P.A. Bolzano|
| 590.9584980237154|            Sardegna|
| 528.6758893280632|              Umbria|
|415.45652173913044|            Calabria|
|264.04545454545456|       Valle d'Aosta|
|205.34387351778656|          Basilicata|
| 178.4822134387352|              

********************************************************************************
Query 10: show top 5 regions for most healed

SELECT 

MAX(dimessi_guariti) AS top_5_regions_for_most_healed, --get max value of dimessi_guariti 

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(dimessi_guariti) DESC --order from the biggest to the smallest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_10=spark.sql("SELECT MAX(dimessi_guariti) AS top_5_regions_for_most_healed, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MAX(dimessi_guariti) DESC LIMIT 5")
query_10.show()

+-----------------------------+--------------+
|top_5_regions_for_most_healed|        region|
+-----------------------------+--------------+
|                       802249|     Lombardia|
|                       411799|      Campania|
|                       409845|        Veneto|
|                       372408|Emilia-Romagna|
|                       350885|      Piemonte|
+-----------------------------+--------------+



********************************************************************************
Query 11: show top 5 regions for less healed

SELECT 

MIN(dimessi_guariti) AS top_5_regions_for_less_healed, --get max value of dimessi_guariti alias top_5_regions_for_less_healed

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(dimessi_guariti) ASC --order from the smallest to the biggest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_11=spark.sql("SELECT MIN(dimessi_guariti) AS top_5_regions_for_less_healed, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(dimessi_guariti) ASC LIMIT 5")
query_11.show()

+-----------------------------+-------------+
|top_5_regions_for_less_healed|       region|
+-----------------------------+-------------+
|                        11210|Valle d'Aosta|
|                        13203|       Molise|
|                        25899|   Basilicata|
|                        44416|  P.A. Trento|
|                        53611|     Sardegna|
+-----------------------------+-------------+



********************************************************************************
Query 12: show average number of healed daily for each region

SELECT 

AVG(dimessi_guariti) AS avg_healed_daily, --get average value of dimessi_guariti alias avg_healed_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(dimessi_guariti) DESC --order from the biggest to the smallest value


In [ ]:
query_12=spark.sql("SELECT AVG(dimessi_guariti) AS avg_healed_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(dimessi_guariti) DESC")
query_12.show(21)

+------------------+--------------------+
|  avg_healed_daily|              region|
+------------------+--------------------+
| 297069.9624505929|           Lombardia|
| 143929.3557312253|              Veneto|
|125870.22134387352|            Piemonte|
|116682.88735177866|      Emilia-Romagna|
|107495.87351778656|            Campania|
| 96871.41501976285|               Lazio|
| 77513.57509881422|             Toscana|
| 63654.11462450593|             Sicilia|
| 61095.96442687747|              Puglia|
|38441.650197628456|             Liguria|
| 33442.66996047431|Friuli Venezia Gi...|
|31922.851778656128|              Marche|
|23620.152173913044|        P.A. Bolzano|
|22018.723320158104|             Abruzzo|
|18967.569169960476|              Umbria|
|16755.013833992096|         P.A. Trento|
| 16630.60276679842|            Calabria|
|14179.754940711462|            Sardegna|
| 6326.442687747036|          Basilicata|
| 4374.353754940711|              Molise|
| 4337.077075098814|       Valle d

********************************************************************************
Query 13: show top 5 regions for most people in intensive care in a single day

SELECT 

MAX(terapia_intensiva) AS top_5_regions_for_most_IC, --get max value of terapia_intensiva alias top_5_regions_for_most_IC

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(terapia_intensiva) DESC --order from the biggest to the smallest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_13=spark.sql("SELECT MAX(terapia_intensiva) AS top_5_regions_for_most_IC, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY MAX(terapia_intensiva) DESC LIMIT 5")
query_13.show(21)

+-------------------------+--------------+
|top_5_regions_for_most_IC|        region|
+-------------------------+--------------+
|                     1381|     Lombardia|
|                      453|      Piemonte|
|                      402|Emilia-Romagna|
|                      398|         Lazio|
|                      372|        Veneto|
+-------------------------+--------------+



********************************************************************************
Query 14: show top 5 regions for less people in intensive care in the last day recorded

SELECT 

MIN(terapia_intensiva) AS top_5_regions_for_less_IC_in_the_last_day, --get min value of terapia_intensiva alias top_5_regions_for_less_IC_in_the_last_day

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(terapia_intensiva) ASC --order from the smallest to the biggest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_14=spark.sql("SELECT MIN(terapia_intensiva) AS top_5_regions_for_less_IC_in_the_last_day, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(terapia_intensiva) ASC LIMIT 5")
query_14.show()

+-----------------------------------------+--------------------+
|top_5_regions_for_less_IC_in_the_last_day|              region|
+-----------------------------------------+--------------------+
|                                        0|       Valle d'Aosta|
|                                        0|          Basilicata|
|                                        0|              Molise|
|                                        0|         P.A. Trento|
|                                        1|Friuli Venezia Gi...|
+-----------------------------------------+--------------------+



********************************************************************************
Query 15: show average number of people in intensive care daily for each region

SELECT 

AVG(terapia_intensiva) AS avg_IC_daily, --get average value of terapia_intensiva alias avg_IC_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(terapia_intensiva) DESC --order from the biggest to the smallest value


In [ ]:
query_15=spark.sql("SELECT AVG(terapia_intensiva) AS avg_IC_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(terapia_intensiva) DESC")
query_15.show(21)

+------------------+--------------------+
|      avg_IC_daily|              region|
+------------------+--------------------+
|398.07707509881425|           Lombardia|
| 158.0592885375494|               Lazio|
|147.72924901185772|      Emilia-Romagna|
|147.51581027667984|            Piemonte|
|125.36561264822134|              Veneto|
|118.43873517786561|             Toscana|
| 87.96837944664031|              Puglia|
| 83.91304347826087|             Sicilia|
| 73.49802371541502|            Campania|
| 49.95652173913044|              Marche|
| 47.58695652173913|             Liguria|
|29.211462450592887|             Abruzzo|
|27.523715415019762|Friuli Venezia Gi...|
|26.954545454545453|              Umbria|
|23.474308300395258|            Sardegna|
| 20.75098814229249|         P.A. Trento|
|14.867588932806324|        P.A. Bolzano|
|14.640316205533598|            Calabria|
| 5.723320158102767|          Basilicata|
| 5.132411067193676|              Molise|
| 4.262845849802371|       Valle d

********************************************************************************
Query 16: show region with most cases in a single day and date

SELECT 

denominazione_regione AS region_with_most_cases_in_a_single_day, --denominazione_regione alias region_with_most_cases_in_a_single_day

data AS date, --data alias date

nuovi_positivi AS new_cases --nuovi_positivi alias new_cases

FROM base --from dpc_covid19_ita_regioni

ORDER BY nuovi_positivi DESC --order from the biggest to the smallest value

LIMIT 1 --limit to 1 to get only the first result


In [ ]:
query_16=spark.sql("SELECT denominazione_regione AS region_with_most_cases_in_a_single_day, data AS date, nuovi_positivi AS new_cases FROM base ORDER BY nuovi_positivi DESC LIMIT 1")
query_16.show()

+--------------------------------------+----------+---------+
|region_with_most_cases_in_a_single_day|      date|new_cases|
+--------------------------------------+----------+---------+
|                             Lombardia|2020-11-07|    11489|
+--------------------------------------+----------+---------+



********************************************************************************
Query 17: show date with biggest increase of cases in Italy and number of cases

SELECT 

data AS date_with_most_cases_in_Italy, --data alias date_with_most_cases_in_Italy

SUM(nuovi_positivi) AS cases --sum nuovi_positivi alias cases

FROM base --from dpc_covid19_ita_regioni

GROUP BY data --group each date

ORDER BY SUM(nuovi_positivi) DESC --order from the biggest to the smallest value

LIMIT 1 --limit to 1 to get only the first result


In [ ]:
query_17=spark.sql("SELECT data AS date_with_biggest_increase_cases_in_Italy, SUM(nuovi_positivi) AS cases FROM base GROUP BY data ORDER BY SUM(nuovi_positivi) DESC LIMIT 1")
query_17.show()

+-----------------------------------------+-----+
|date_with_biggest_increase_cases_in_Italy|cases|
+-----------------------------------------+-----+
|                               2020-11-13|40902|
+-----------------------------------------+-----+



********************************************************************************
Query 18: show biggest increase of cases in a single region and date

SELECT 

variazione_totale_positivi AS biggest_increase_of_cases, --variazione_totale_positivi alias biggest_increase_of_cases

denominazione_regione AS region, --denominazione_regione alias region

data AS date --data alias date

FROM base --from dpc_covid19_ita_regioni

ORDER BY variazione_totale_positivi DESC --order from the biggest to the smallest value

LIMIT 1 --limit to 1 to get only the first result


In [ ]:
query_18=spark.sql("SELECT variazione_totale_positivi AS biggest_increase_of_cases, denominazione_regione AS region, data AS date FROM base ORDER BY variazione_totale_positivi DESC LIMIT 1")
query_18.show()

+-------------------------+---------+----------+
|biggest_increase_of_cases|   region|      date|
+-------------------------+---------+----------+
|                    10263|Lombardia|2020-11-07|
+-------------------------+---------+----------+



********************************************************************************
Query 19: show biggest decrease of cases in a single region and date

SELECT 

variazione_totale_positivi AS biggest_decrease_of_cases, --variazione_totale_positivi alias biggest_decrease_of_cases

denominazione_regione AS region, --denominazione_regione alias region

data AS date --data alias date

FROM base --from dpc_covid19_ita_regioni

ORDER BY variazione_totale_positivi ASC --order from the smallest to the biggest value

LIMIT 1 --limit to 1 to get only the first result


In [ ]:
query_19=spark.sql("SELECT variazione_totale_positivi AS biggest_decrease_of_cases, denominazione_regione AS region, data AS date FROM base ORDER BY variazione_totale_positivi ASC LIMIT 1")
query_19.show()

+-------------------------+--------+----------+
|biggest_decrease_of_cases|  region|      date|
+-------------------------+--------+----------+
|                   -48092|Campania|2021-06-15|
+-------------------------+--------+----------+



********************************************************************************
Query 20: show top 5 regions for most people in home isolation of all time

SELECT 

MAX(isolamento_domiciliare) AS top_5_regions_for_most_home_isolation_of_all_time, --get max value of isolamento_domiciliare alias top_5_regions_for_most_home_isolation_of_all_time

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY MAX(isolamento_domiciliare) DESC --order from the biggest to the smallest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_20=spark.sql("SELECT MAX(isolamento_domiciliare) AS top_5_regions_for_most_home_isolation_of_all_time, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY MAX(isolamento_domiciliare) DESC LIMIT 5")
query_20.show()

+-------------------------------------------------+---------+
|top_5_regions_for_most_home_isolation_of_all_time|   region|
+-------------------------------------------------+---------+
|                                           155066|Lombardia|
|                                           102438| Campania|
|                                           101041|   Veneto|
|                                            90735|    Lazio|
|                                            73011| Piemonte|
+-------------------------------------------------+---------+



********************************************************************************
Query 21: show top 5 regions for less people in home isolation in the last day 
recorded
SELECT 

MIN(isolamento_domiciliare) AS top_5_regions_for_less_home_isolation_in_the_last_day, --get min value of isolamento_domiciliare alias top_5_regions_for_less_home_isolation_in_the_last_day

denominazione_regione AS region --denominazione_regione alias region

FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) --from rows with the last date recorded

GROUP BY denominazione_regione --group each region name

ORDER BY MIN(isolamento_domiciliare) ASC --order from the smallest to the biggest value

LIMIT 5 --limit to 5 to get only the first 5 results


In [ ]:
query_21=spark.sql("SELECT MIN(isolamento_domiciliare) AS top_5_regions_for_less_home_isolation_in_the_last_day, denominazione_regione AS region FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) GROUP BY denominazione_regione ORDER BY MIN(isolamento_domiciliare) ASC LIMIT 5")
query_21.show()

+-----------------------------------------------------+-------------+
|top_5_regions_for_less_home_isolation_in_the_last_day|       region|
+-----------------------------------------------------+-------------+
|                                                   19|Valle d'Aosta|
|                                                   49|  P.A. Trento|
|                                                   54|       Molise|
|                                                  131| P.A. Bolzano|
|                                                  168|      Liguria|
+-----------------------------------------------------+-------------+



********************************************************************************
Query 22: show average number of people in home isolation daily for each region

SELECT 

AVG(isolamento_domiciliare) AS avg_home_isolation_daily, --get average value of isolamento_domiciliare alias avg_home_isolation_daily

denominazione_regione AS region --denominazione_regione alias region

FROM base --from dpc_covid19_ita_regioni

GROUP BY denominazione_regione --group each region name

ORDER BY AVG(isolamento_domiciliare) DESC --order from the biggest to the smallest value


In [ ]:
query_22=spark.sql("SELECT AVG(isolamento_domiciliare) AS avg_home_isolation_daily, denominazione_regione AS region FROM base GROUP BY denominazione_regione ORDER BY AVG(isolamento_domiciliare) DESC")
query_22.show(21)

+------------------------+--------------------+
|avg_home_isolation_daily|              region|
+------------------------+--------------------+
|       37699.69565217391|            Campania|
|       37060.92490118577|           Lombardia|
|       24552.12648221344|               Lazio|
|       23227.31818181818|      Emilia-Romagna|
|      20904.098814229248|              Veneto|
|      18647.203557312252|              Puglia|
|       13400.05138339921|            Piemonte|
|      12721.478260869566|             Sicilia|
|      10142.707509881422|             Toscana|
|       6843.333992094862|            Sardegna|
|       5001.559288537549|             Abruzzo|
|       4881.875494071146|Friuli Venezia Gi...|
|       4552.646245059289|              Marche|
|       4539.940711462451|            Calabria|
|      3159.2747035573125|             Liguria|
|       2832.620553359684|        P.A. Bolzano|
|      2460.2648221343875|              Umbria|
|       2288.498023715415|          Basi

********************************************************************************
Query 23: show top 5 regions for total cases and number of first doses, second doses and total doses of vaccine done

--td total doses

--lr last record

--sd sum doses

--fsd first second doses


SELECT 

td.region, --region name from td

td.total_cases, --total cases from td

fsd.first_dose, --first dose from fsd

fsd.second_dose, --second dose from fsd

td.total_doses --total doses from td

FROM 

	(SELECT 

	base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

	SUM(base2.prima_dose) AS first_dose, --sum prima_dose from somministrazioni_vaccini_latest alias first_dose

	SUM(base2.seconda_dose) AS second_dose --sum seconda_dose from somministrazioni_vaccini_latest alias second_dose

	FROM base --from dpc_covid19_ita_regioni

	JOIN base2 --inner join somministrazioni_vaccini_latest

	ON base.data=base2.data_somministrazione --matching based on dates

	AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

	GROUP BY base.denominazione_regione) AS fsd --group each region name, alias fsd

JOIN --inner join

	(SELECT 

	lr.denominazione_regione AS region, --denominazione_regione from lr alias region

	lr.totale_casi AS total_cases, --totale_casi from lr alias total_cases

	sd.dosi AS total_doses --dosi from sd alias total_doses

	FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr --from rows with the last date recorded, alias lr

	JOIN --inner join

		(SELECT 

		base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

		SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi --sum sums of prima_dose and seconda_dose alias dosi

		FROM base --from dpc_covid19_ita_regioni

		JOIN base2 --inner join somministrazioni_vaccini_latest

		ON base.data=base2.data_somministrazione --matching based on dates

		AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

		GROUP BY base.denominazione_regione) AS sd --group each region name, alias sd

	ON lr.denominazione_regione=sd.denominazione_regione --matching based on region names

	ORDER BY lr.totale_casi DESC --order from biggest to smallest value

	LIMIT 5) AS td --limit to 5 to get only the first 5 results, alias td

ON td.region=fsd.denominazione_regione --matching based on region names

ORDER BY td.total_cases DESC --order from biggest to smallest value


In [ ]:
query_23=spark.sql("SELECT td.region, td.total_cases, fsd.first_dose, fsd.second_dose, td.total_doses FROM (SELECT base.denominazione_regione, SUM(base2.prima_dose) AS first_dose, SUM(base2.seconda_dose) AS second_dose FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS fsd JOIN (SELECT lr.denominazione_regione AS region, lr.totale_casi AS total_cases, sd.dosi AS total_doses FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr JOIN (SELECT base.denominazione_regione, SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS sd ON lr.denominazione_regione=sd.denominazione_regione ORDER BY lr.totale_casi DESC LIMIT 5) AS td ON td.region=fsd.denominazione_regione ORDER BY td.total_cases DESC")
query_23.show()

+--------------+-----------+----------+-----------+-----------+
|        region|total_cases|first_dose|second_dose|total_doses|
+--------------+-----------+----------+-----------+-----------+
|     Lombardia|     843891|   6441249|    3832843|   10274092|
|        Veneto|     426743|   2748421|    1879097|    4627518|
|      Campania|     426122|   3357789|    2306355|    5664144|
|Emilia-Romagna|     387789|   2573969|    1698010|    4271979|
|      Piemonte|     363360|   2529698|    1620581|    4150279|
+--------------+-----------+----------+-----------+-----------+



********************************************************************************
Query 24: show top 5 regions for total healed and number of first doses, second doses and total doses of vaccine done

--td total doses

--lr last record

--sd sum doses

--fsd first second doses


SELECT 

td.region, --region name from td

td.total_healed, --total healed from td

fsd.first_dose, --first dose from fsd

fsd.second_dose, --second dose from fsd

td.total_doses --total doses from td

FROM 

	(SELECT 

	base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

	SUM(base2.prima_dose) AS first_dose, --sum prima_dose from somministrazioni_vaccini_latest alias first_dose

	SUM(base2.seconda_dose) AS second_dose --sum seconda_dose from somministrazioni_vaccini_latest alias second_dose

	FROM base --from dpc_covid19_ita_regioni

	JOIN base2 --inner join somministrazioni_vaccini_latest

	ON base.data=base2.data_somministrazione --matching based on dates

	AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

	GROUP BY base.denominazione_regione) AS fsd --group each region name, alias fsd

JOIN --inner join

	(SELECT 

	lr.denominazione_regione AS region, --denominazione_regione from lr alias region

	lr.dimessi_guariti AS total_healed, --dimessi_guariti from lr alias total_healed

	sd.dosi AS total_doses --dosi from sd alias total_doses

	FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr --from rows with the last date recorded, alias lr

	JOIN --inner join

		(SELECT 

		base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

		SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi --sum sums of prima_dose and seconda_dose alias dosi

		FROM base --from dpc_covid19_ita_regioni

		JOIN base2 --inner join somministrazioni_vaccini_latest

		ON base.data=base2.data_somministrazione --matching based on dates

		AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

		GROUP BY base.denominazione_regione) AS sd --group each region name, alias sd

	ON lr.denominazione_regione=sd.denominazione_regione --matching based on region names

	ORDER BY lr.totale_casi DESC --order from biggest to smallest value

	LIMIT 5) AS td --limit to 5 to get only the first 5 results, alias td

ON td.region=fsd.denominazione_regione --matching based on region names

ORDER BY td.total_healed DESC --order from biggest to smallest value


In [ ]:
query_24=spark.sql("SELECT td.region, td.total_healed, fsd.first_dose, fsd.second_dose, td.total_doses FROM (SELECT base.denominazione_regione, SUM(base2.prima_dose) AS first_dose, SUM(base2.seconda_dose) AS second_dose FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS fsd JOIN (SELECT lr.denominazione_regione AS region, lr.dimessi_guariti AS total_healed, sd.dosi AS total_doses FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr JOIN (SELECT base.denominazione_regione, SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS sd ON lr.denominazione_regione=sd.denominazione_regione ORDER BY lr.totale_casi DESC LIMIT 5) AS td ON td.region=fsd.denominazione_regione ORDER BY td.total_healed DESC")
query_24.show()

+--------------+------------+----------+-----------+-----------+
|        region|total_healed|first_dose|second_dose|total_doses|
+--------------+------------+----------+-----------+-----------+
|     Lombardia|      802249|   6441249|    3832843|   10274092|
|      Campania|      411799|   3357789|    2306355|    5664144|
|        Veneto|      409845|   2748421|    1879097|    4627518|
|Emilia-Romagna|      372408|   2573969|    1698010|    4271979|
|      Piemonte|      350885|   2529698|    1620581|    4150279|
+--------------+------------+----------+-----------+-----------+



********************************************************************************
Query 25: show top 5 regions for total deaths and number of first doses, second doses and total doses of vaccine done

SELECT 

td.region, --region name from td

td.total_deaths, --total deaths from td

fsd.first_dose, --first dose from fsd

fsd.second_dose, --second dose from fsd

td.total_doses --total doses from td

FROM 

	(SELECT 

	base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

	SUM(base2.prima_dose) AS first_dose, --sum prima_dose from somministrazioni_vaccini_latest alias first_dose

	SUM(base2.seconda_dose) AS second_dose --sum seconda_dose from somministrazioni_vaccini_latest alias second_dose

	FROM base --from dpc_covid19_ita_regioni

	JOIN base2 --inner join somministrazioni_vaccini_latest

	ON base.data=base2.data_somministrazione --matching based on dates

	AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

	GROUP BY base.denominazione_regione) AS fsd --group each region name, alias fsd

JOIN --inner join

	(SELECT 

	lr.denominazione_regione AS region, --denominazione_regione from lr alias region

	lr.deceduti AS total_deaths, --deceduti from lr alias total_deaths

	sd.dosi AS total_doses --dosi from sd alias total_doses

	FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr --from rows with the last date recorded, alias lr

	JOIN --inner join

		(SELECT 

		base.denominazione_regione, --denominazione_regione from dpc_covid19_ita_regioni

		SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi --sum sums of prima_dose and seconda_dose alias dosi

		FROM base --from dpc_covid19_ita_regioni

		JOIN base2 --inner join somministrazioni_vaccini_latest

		ON base.data=base2.data_somministrazione --matching based on dates

		AND base.codice_regione=base2.codice_regione_ISTAT --and matching based on region codes

		GROUP BY base.denominazione_regione) AS sd --group each region name, alias sd

	ON lr.denominazione_regione=sd.denominazione_regione --matching based on region names

	ORDER BY lr.totale_casi DESC --order from biggest to smallest value

	LIMIT 5) AS td --limit to 5 to get only the first 5 results, alias td

ON td.region=fsd.denominazione_regione --matching based on region names

ORDER BY td.total_deaths DESC --order from biggest to smallest value


In [ ]:
query_25=spark.sql("SELECT td.region, td.total_deaths, fsd.first_dose, fsd.second_dose, td.total_doses FROM (SELECT base.denominazione_regione, SUM(base2.prima_dose) AS first_dose, SUM(base2.seconda_dose) AS second_dose FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS fsd JOIN (SELECT lr.denominazione_regione AS region, lr.deceduti AS total_deaths, sd.dosi AS total_doses FROM (SELECT * FROM base ORDER BY data DESC LIMIT 21) AS lr JOIN (SELECT base.denominazione_regione, SUM(base2.prima_dose)+SUM(base2.seconda_dose) AS dosi FROM base JOIN base2 ON base.data=base2.data_somministrazione AND base.codice_regione=base2.codice_regione_ISTAT GROUP BY base.denominazione_regione) AS sd ON lr.denominazione_regione=sd.denominazione_regione ORDER BY lr.totale_casi DESC LIMIT 5) AS td ON td.region=fsd.denominazione_regione ORDER BY td.total_deaths DESC")
query_25.show()

+--------------+------------+----------+-----------+-----------+
|        region|total_deaths|first_dose|second_dose|total_doses|
+--------------+------------+----------+-----------+-----------+
|     Lombardia|       33802|   6441249|    3832843|   10274092|
|Emilia-Romagna|       13268|   2573969|    1698010|    4271979|
|      Piemonte|       11698|   2529698|    1620581|    4150279|
|        Veneto|       11623|   2748421|    1879097|    4627518|
|      Campania|        7551|   3357789|    2306355|    5664144|
+--------------+------------+----------+-----------+-----------+



********************************************************************************
Query 26: show for each vaccine the number of first doses, second doses and total doses done

SELECT 

fornitore AS vaccine, --fornitore as vaccine

SUM(prima_dose) AS first_dose, --sum prima_dose alias first_dose

SUM(seconda_dose) AS second_dose, --sum seconda_dose alias second_dose

SUM(prima_dose)+SUM(seconda_dose) AS total_doses --sum sums of prima_dose and seconda_dose alias total_doses

FROM base2 --from somministrazioni_vaccini_latest

GROUP BY fornitore --group each vaccine name


In [ ]:
query_26=spark.sql("SELECT fornitore AS vaccine, SUM(prima_dose) AS first_dose, SUM(seconda_dose) AS second_dose, SUM(prima_dose)+SUM(seconda_dose) AS total_doses FROM base2 GROUP BY fornitore")
query_26.show()

+--------------------+----------+-----------+-----------+
|             vaccine|first_dose|second_dose|total_doses|
+--------------------+----------+-----------+-----------+
|Vaxzevria (AstraZ...|   6339847|    3605565|    9945412|
|             Janssen|   1276592|          0|    1276592|
|             Moderna|   3267167|    2362126|    5629293|
|     Pfizer/BioNTech|  24381186|   16779533|   41160719|
+--------------------+----------+-----------+-----------+



********************************************************************************
Query 27: show for each region, for each vaccine the number of first doses, second doses and total doses done

SELECT 

nome_area AS region, --nome_area alias region

codice_regione_ISTAT AS reg_cod, --codice_regione_ISTAT alias reg_cod

fornitore AS vaccine, --fornitore as vaccine

SUM(prima_dose) AS first_dose, --sum prima_dose alias first_dose

SUM(seconda_dose) AS second_dose, --sum seconda_dose alias second_dose

SUM(prima_dose)+SUM(seconda_dose) AS total_doses --sum sums of prima_dose and seconda_dose alias total_doses

FROM base2 --from somministrazioni_vaccini_latest

GROUP BY nome_area, codice_regione_ISTAT, fornitore --group each region name, region code and vaccine name

ORDER BY codice_regione_ISTAT,fornitore --order from the smallest to the biggest value by default codice_regione_ISTAT then fornitore


In [ ]:
query_27=spark.sql("SELECT area AS region, codice_regione_ISTAT AS reg_cod, fornitore AS vaccine, SUM(prima_dose) AS first_dose, SUM(seconda_dose) AS second_dose, SUM(prima_dose)+SUM(seconda_dose) AS total_doses FROM base2 GROUP BY area, codice_regione_ISTAT, fornitore ORDER BY codice_regione_ISTAT,fornitore")
query_27.show(84)

+------+-------+--------------------+----------+-----------+-----------+
|region|reg_cod|             vaccine|first_dose|second_dose|total_doses|
+------+-------+--------------------+----------+-----------+-----------+
|   PIE|      1|             Janssen|     63923|          0|      63923|
|   PIE|      1|             Moderna|    232157|     158403|     390560|
|   PIE|      1|     Pfizer/BioNTech|   1825357|    1199233|    3024590|
|   PIE|      1|Vaxzevria (AstraZ...|    408261|     262945|     671206|
|   VDA|      2|             Janssen|       968|          0|        968|
|   VDA|      2|             Moderna|      4876|       4518|       9394|
|   VDA|      2|     Pfizer/BioNTech|     49633|      36124|      85757|
|   VDA|      2|Vaxzevria (AstraZ...|     13137|       6040|      19177|
|   LOM|      3|             Janssen|    266389|          0|     266389|
|   LOM|      3|             Moderna|    586415|     414466|    1000881|
|   LOM|      3|     Pfizer/BioNTech|   4383737|   

********************************************************************************
Query 28: show the number of doses administered to males, females and the total number

SELECT 

SUM(sesso_maschile) AS males, --sum sesso_maschile alias males

SUM(sesso_femminile) AS females, --sum sesso_femminile alias females

SUM(sesso_maschile)+SUM(sesso_femminile) AS combined --sum sums of sesso_maschile and sesso_femminile alias combined

FROM base2 -- from somministrazioni_vaccini_latest


In [ ]:
query_28=spark.sql("SELECT SUM(sesso_maschile) AS males, SUM(sesso_femminile) AS females, SUM(sesso_maschile)+SUM(sesso_femminile) AS combined FROM base2")
query_28.show()

+--------+--------+--------+
|   males| females|combined|
+--------+--------+--------+
|27427183|31362716|58789899|
+--------+--------+--------+



********************************************************************************
Query 29: show for each region the number of doses administered to males, 
females and the total number

SELECT 

nome_area AS region, --nome_area alias region

SUM(sesso_maschile) AS total_doses_received_males, --sum sesso_maschile alias total_doses_received_males 

SUM(sesso_femminile) AS total_doses_received_females, --sum sesso_femminile alias total_doses_received_females

SUM(sesso_maschile)+SUM(sesso_femminile) AS total_doses_received --sum of sums of sesso_maschile and sesso_femminile alias total_doses_received

FROM base2 --from somministrazioni_vaccini_latest

GROUP BY nome_area --group each region name

ORDER BY total_doses_received DESC --order from the biggest to the smallest value


In [ ]:
query_29=spark.sql("SELECT area AS region, SUM(sesso_maschile) AS total_doses_received_males, SUM(sesso_femminile) AS total_doses_received_females, SUM(sesso_maschile)+SUM(sesso_femminile) AS total_doses_received FROM base2 GROUP BY area ORDER BY total_doses_received DESC")
query_29.show(22)

+------+--------------------------+----------------------------+--------------------+
|region|total_doses_received_males|total_doses_received_females|total_doses_received|
+------+--------------------------+----------------------------+--------------------+
|   LOM|                   4875449|                     5564048|            10439497|
|   LAZ|                   2767637|                     3216792|             5984429|
|   CAM|                   2732054|                     3013501|             5745555|
|   VEN|                   2225489|                     2517787|             4743276|
|   SIC|                   2046789|                     2302374|             4349163|
|   EMR|                   1991478|                     2328145|             4319623|
|   PIE|                   1939871|                     2251379|             4191250|
|   PUG|                   1885143|                     2171999|             4057142|
|   TOS|                   1583177|                   

********************************************************************************
Query 30: show for each age group the number of first doses, second doses and total doses done

SELECT 

fascia_anagrafica AS age_group, --fascia_anagrafica alias age_group

SUM(prima_dose) AS first_dose, --sum prima_dose alias first_dose

SUM(seconda_dose) AS second_dose, --sum seconda_dose alias second_dose

SUM(prima_dose)+SUM(seconda_dose) AS total_doses --sum sums of prima_dose and seconda_dose alias total_doses

FROM base2 --from somministrazioni_vaccini_latest

GROUP BY fascia_anagrafica --group each age group

ORDER BY fascia_anagrafica --order from the smallest to the biggest value by default


In [ ]:
query_30=spark.sql("SELECT fascia_anagrafica AS age_group, SUM(prima_dose) AS first_dose, SUM(seconda_dose) AS second_dose, SUM(prima_dose)+SUM(seconda_dose) AS total_doses FROM base2 GROUP BY fascia_anagrafica ORDER BY fascia_anagrafica")
query_30.show()

+---------+----------+-----------+-----------+
|age_group|first_dose|second_dose|total_doses|
+---------+----------+-----------+-----------+
|    12-19|   1151003|     274188|    1425191|
|    20-29|   2998593|    1079569|    4078162|
|    30-39|   3490333|    1437616|    4927949|
|    40-49|   5378490|    2801931|    8180421|
|    50-59|   6906355|    4869320|   11775675|
|    60-69|   6024298|    4057641|   10081939|
|    70-79|   5168175|    4238675|    9406850|
|    80-89|   3369692|    3254089|    6623781|
|      90+|    777853|     734195|    1512048|
+---------+----------+-----------+-----------+



********************************************************************************
Query 31: show for each region, for each age group the number of first doses, second doses and total doses done

SELECT 

nome_area AS region, --nome_area alias region

fascia_anagrafica AS age_group, --fascia_anagrafica alias age_group

SUM(prima_dose) AS first_dose, --sum prima_dose alias first_dose

SUM(seconda_dose) AS second_dose, --sum seconda_dose alias second_dose

SUM(prima_dose)+SUM(seconda_dose) AS total_doses --sum sums of prima_dose and seconda_dose alias total_doses

FROM base2 --from somministrazioni_vaccini_latest

GROUP BY nome_area, fascia_anagrafica --group each region name then each age group

ORDER BY nome_area, fascia_anagrafica --order alphabetically region names then from the smallest to the biggest value the age groups


In [ ]:
query_31=spark.sql("SELECT area AS region, fascia_anagrafica AS age_group, SUM(prima_dose) AS first_dose, SUM(seconda_dose) AS second_dose, SUM(prima_dose)+SUM(seconda_dose) AS total_doses FROM base2 GROUP BY area, fascia_anagrafica ORDER BY area, fascia_anagrafica")
query_31.show(189)

+------+---------+----------+-----------+-----------+
|region|age_group|first_dose|second_dose|total_doses|
+------+---------+----------+-----------+-----------+
|   ABR|    12-19|     27406|       8052|      35458|
|   ABR|    20-29|     68700|      23940|      92640|
|   ABR|    30-39|     74252|      30558|     104810|
|   ABR|    40-49|    117454|      62934|     180388|
|   ABR|    50-59|    149407|      92631|     242038|
|   ABR|    60-69|    138628|      90757|     229385|
|   ABR|    70-79|    116577|      83242|     199819|
|   ABR|    80-89|     75638|      73331|     148969|
|   ABR|      90+|     18348|      17617|      35965|
|   BAS|    12-19|      8684|       2061|      10745|
|   BAS|    20-29|     22716|      11168|      33884|
|   BAS|    30-39|     26034|      14883|      40917|
|   BAS|    40-49|     48902|      23446|      72348|
|   BAS|    50-59|     63362|      35293|      98655|
|   BAS|    60-69|     61389|      34002|      95391|
|   BAS|    70-79|     47095